To-do:
* Table of contents
* Introduction
* Conclusion
* GitHub write-up

# Predicting LendingClub Loan Charge-offs from Application Data

***By Joe Corliss***

**June 10, 2018**

## Table of Contents

1. [Summary](#1)
    1. [Spoilers](#1.1)

# Introduction

LendingClub is a US peer-to-peer lending company and the world's largest peer-to-peer lending platform. As explained by [Wikipedia](https://en.wikipedia.org/wiki/Lending_Club),

> Lending Club enables borrowers to create unsecured personal loans between \$1,000 and \$40,000. The standard loan period is three years. Investors can search and browse the loan listings on Lending Club website and select loans that they want to invest in based on the information supplied about the borrower, amount of loan, loan grade, and loan purpose. Investors make money from interest. Lending Club makes money by charging borrowers an origination fee and investors a service fee.

The goal of this project is to predict loan charge-offs only using data available at the time of the loan application.

## Sources
* LendingClub
 * [LendingClub website](https://www.lendingclub.com/)
 * [LendingClub statistics](https://www.lendingclub.com/info/download-data.action) - Original data source
 * [Wikipedia page](https://en.wikipedia.org/wiki/Lending_Club)
* Data
 * [All Lending Club loan data](https://www.kaggle.com/wordsforthewise/lending-club) - The dataset used in this project, hosted on Kaggle
 * [Lending Club Loan Data](https://www.kaggle.com/wendykan/lending-club-loan-data) - Another LendingClub dataset on Kaggle, not used in this project
* This project
 * [Kaggle notebook](https://www.kaggle.com/pileatedperch/predicting-loan-status-mcc-0-73)
 * [GitHub repo](https://github.com/jgcorliss/lending-club)

# Import the Data
<a id="2"></a>

In [3]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Pandas options
pd.set_option('display.max_colwidth', 1000, 'display.max_rows', None, 'display.max_columns', None)

# Plotting options
%matplotlib inline
mpl.style.use('ggplot')
sns.set(style='whitegrid')

Read the data into a pandas dataframe:

In [6]:
loans = pd.read_csv('accepted_2007_to_2017Q3.csv.gz', compression='gzip', low_memory=True)

C:\Users\Joe\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (0,18,48,58,117,128,129,130,133,134,135,138,144,145,146) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Check basic dataframe info:

In [7]:
loans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1646801 entries, 0 to 1646800
Columns: 150 entries, id to settlement_term
dtypes: float64(113), object(37)
memory usage: 1.8+ GB


There are 1.6 million rows and 150 variables. The size of the dataset is 1.8 GB.

Let's peek at 5 randomly selected rows. Each row corresponds to a single loan.

In [8]:
loans.sample(5)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
1068072,96830469,NaN,8625.0,8625.0,8625.0,36 months,13.49,292.66,C,C2,Data Manager,3 years,RENT,61700.0,Verified,Jan-2017,Fully Paid,n,NaN,credit_card,Credit card refinancing,303xx,GA,31.80,0.0,Apr-2003,685.0,689.0,0.0,47.0,NaN,13.0,0.0,13698.0,43.5,50.0,w,0.00,0.00,9059.195658,9059.20,8625.00,434.20,0.0,0.0,0.0,Jun-2017,7916.91,NaN,Dec-2017,694.0,690.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,157198.0,0.0,4.0,1.0,5.0,8.0,143500.0,82.0,0.0,2.0,4055.0,68.0,31500.0,0.0,2.0,0.0,7.0,13100.0,5356.0,55.7,0.0,0.0,165.0,142.0,22.0,8.0,0.0,41.0,47.0,22.0,47.0,0.0,2.0,5.0,2.0,4.0,34.0,8.0,15.0,5.0,13.0,0.0,0.0,0.0,1.0,100.0,0.0,0.0,0.0,168498.0,157198.0,12100.0,136469.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1477618,91688305,NaN,11200.0,11200.0,11200.0,36 months,8.24,352.21,B,B1,Consultant,< 1 year,MORTGAGE,70400.0,Source Verified,Nov-2016,Current,n,NaN,debt_consolidation,Debt consolidation,193xx,PA,16.33,1.0,Mar-2005,720.0,724.0,2.0,10.0,NaN,18.0,0.0,12668.0,22.6,32.0,f,7469.83,7469.83,4568.480000,4568.48,3730.17,838.31,0.0,0.0,0.0,Dec-2017,352.21,Jan-2018,Dec-2017,674.0,670.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,237358.0,3.0,5.0,1.0,1.0,9.0,23815.0,74.0,4.0,5.0,2129.0,41.0,56000.0,1.0,0.0,4.0,6.0,13187.0,26644.0,16.5,0.0,0.0,121.0,139.0,1.0,1.0,1.0,5.0,50.0,1.0,10.0,0.0,3.0,6.0,4.0,8.0,7.0,12.0,24.0,6.0,18.0,0.0,0.0,0.0,5.0,93.8,0.0,0.0,0.0,306325.0,36483.0,31900.0,32325.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

# Response Variable
<a id="3"></a>

We're going to try to predict the `loan_status` variable. What are the value counts for this variable?

In [9]:
loans['loan_status'].value_counts(dropna=False)

Current                                                788950
Fully Paid                                             646902
Charged Off                                            168084
Late (31-120 days)                                      23763
In Grace Period                                         10474
Late (16-30 days)                                        5786
Does not meet the credit policy. Status:Fully Paid       1988
Does not meet the credit policy. Status:Charged Off       761
Default                                                    70
NaN                                                        23
Name: loan_status, dtype: int64

We're going to try to learn differences in the features between completed loans that have been fully paid or charged off. We won't consider loans that are current, don't meet the credit policy, defaulted, or have a missing status. So we only keep the loans with status "Fully Paid" or "Charged Off."

In [10]:
loans = loans.loc[loans['loan_status'].isin(['Fully Paid', 'Charged Off'])]

How many loans remain in the dataset?

In [11]:
loans.shape

(814986, 150)

There are 814,986 loans remaining. Check that the statuses are as expected:

In [12]:
loans['loan_status'].value_counts(dropna=False)

Fully Paid     646902
Charged Off    168084
Name: loan_status, dtype: int64

Status counts as percentages:

In [13]:
loans['loan_status'].value_counts(normalize=True, dropna=False)

Fully Paid     0.793758
Charged Off    0.206242
Name: loan_status, dtype: float64

About 79% of the remaining loans have been fully paid and 21% have charged off, so we have a somewhat unbalanced classification problem.

In [18]:
loans[['sub_grade', 'int_rate']].sample(20)

,sub_grade,int_rate
176181,B2,10.99
456708,D4,17.57
243298,C1,12.99
694184,A2,6.49
261204,A5,7.88
950341,B4,13.11
260781,C2,13.61
1646740,D3,16.99
586951,C4,13.99
1644277,B1,8.49


# Feature Selection

The dataset has 150 features, but we'll choose to neglect some features in predicting the loan status. We'll first drop any features that meet either of the following criteria:

1. More than 30% of the data is missing (in the training set)
2. The feature would not have been available at the time of the loan application

Definitions of the features are given in the Lending Club Data Dictionary [available here](https://www.lendingclub.com/info/download-data.action).

## Drop features missing too much data
<a id="4.2"></a>

First we calculate the percentage of missing data for each feature:

In [ ]:
missing_fractions = loans.isnull().mean().sort_values(ascending=False)

Let's visualize the distribution of missing data percentages:

In [ ]:
plt.figure(figsize=(6,3), dpi=90)
missing_fractions.plot.hist(bins=20)
plt.title('Histogram of Variable Incompleteness')
plt.xlabel('Fraction of data missing')
plt.ylabel('Variable count')

From the above histogram, we see there's a large gap between features missing "some" data and those missing "lots" of data. Because it's generally very difficult to accurately impute data with more than 30% missing values, we drop such columns. First store all variables missing more than 30% data in a list:

In [ ]:
drop_list = sorted(list(missing_fractions[missing_fractions > 0.3].index))
print(drop_list)

How many features will be dropped?

In [ ]:
len(drop_list)

Now drop these features.

In [ ]:
loans.drop(labels=drop_list, axis=1, inplace=True)

## Remove features that were unavailable before the loan was funded
<a id="4.4"></a>

We examine the Lending Club Data Dictionary to determine which features would not have been available before the loan was funded. The idea here is to predict whether a loan will be paid off **before** deciding to fund the loan. Here's the list of features we currently have, in alphabetical order:

In [ ]:
print(sorted(list(loans.columns)))

For each of these features, we check the description in the Data Dictionary, and drop those features that we believe would not have been available before the loan was funded. When in doubt, we err on the side of dropping the feature.

In [ ]:
drop_list = ['acc_now_delinq', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'collection_recovery_fee', 'collections_12_mths_ex_med', 'debt_settlement_flag', 'delinq_2yrs', 'delinq_amnt', 'disbursement_method', 'funded_amnt', 'funded_amnt_inv', 'hardship_flag', 'inq_last_6mths', 'last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low', 'last_pymnt_amnt', 'last_pymnt_d', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mths_since_recent_bc', 'mths_since_recent_inq', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'open_acc', 'out_prncp', 'out_prncp_inv', 'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'pymnt_plan', 'recoveries', 'revol_bal', 'revol_util', 'tax_liens', 'tot_coll_amt', 'tot_cur_bal', 'tot_hi_cred_lim', 'total_acc', 'total_bal_ex_mort', 'total_bc_limit', 'total_il_high_credit_limit', 'total_pymnt', 'total_pymnt_inv', 'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp', 'total_rev_hi_lim']

How many features will be dropped?

In [ ]:
len(drop_list)

Drop these features:

In [ ]:
loans.drop(labels=drop_list, axis=1, inplace=True)

## Inspect the remaining features

We'll inspect each feature individually, and do the following:

1. View summary statistics and visualize the data, if desired.
2. Modify the feature to make it useful for modeling, if necessary.
3. Drop the feature if it can't be made useful for modeling.

Define a plotting utility:

In [ ]:
def plot_var(col_name, full_name, continuous):
    """
    Visualize a variable with and without faceting on the loan status.
    - col_name is the variable name in the dataframe
    - full_name is the full variable name
    - continuous is True if the variable is continuous, False otherwise
    """
    f, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(12,3), dpi=90)
    
    # Plot without loan status
    if continuous:
        sns.distplot(loans.loc[loans[col_name].notnull(), col_name], kde=False, ax=ax1)
    else:
        sns.countplot(loans[col_name], order=sorted(loans[col_name].unique()), color='#5975A4', saturation=1, ax=ax1)
    ax1.set_xlabel(full_name)
    ax1.set_ylabel('Count')
    ax1.set_title(full_name)

    # Plot with loan status
    if continuous:
        sns.boxplot(x=col_name, y='loan_status', data=loans, ax=ax2)
        ax2.set_ylabel('')
        ax2.set_title(full_name + ' by Loan Status')
    else:
        charge_off_rates = loans.groupby(col_name)['loan_status'].value_counts(normalize=True).loc[:,'Charged Off']
        sns.barplot(x=charge_off_rates.index, y=charge_off_rates.values, color='#5975A4', saturation=1, ax=ax2)
        ax2.set_ylabel('Fraction of Loans Charged Off')
        ax2.set_title('Charge Off Rate by ' + full_name)
    ax2.set_xlabel(full_name)
    
    plt.tight_layout()

How many features are left?

In [ ]:
loans.shape[1]

What are the remaining features?

In [ ]:
print(list(loans.columns))

### id

Data Dictionary: "A unique [Lending Club] assigned ID for the loan listing."

In [ ]:
loans['id'].sample(5)

 Are all the IDs in fact unique?

In [ ]:
loans['id'].describe()

The ID is not useful for modeling, either as a categorical variable (there are too many distinct values) or as a numerical variable (the IDs vary wildly in magnitude, likely without any significance), so we drop this variable.

In [ ]:
loans.drop('id', axis=1, inplace=True)

### loan_amnt

Data Dictionary: "The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value."

In [ ]:
loans['loan_amnt'].describe()

Loan amounts range from \$500 to \$40,000, with a median of \$12,000.

In [ ]:
plot_var('loan_amnt', 'Loan Amounts', continuous=True)

Charged-off loans tend to have higher loan amounts. Because of the large number of samples, the difference in the medians is likely to be statistically significant.

### term

Data Dictionary: "The number of payments on the loan. Values are in months and can be either 36 or 60."

Let's check the value counts:

In [ ]:
loans['term'].value_counts(dropna=False)

Convert `term` to integers.

In [ ]:
loans.loc[:,'term'] = loans['term'].apply(lambda s: np.int8(s.split()[0]))

In [ ]:
loans['term'].value_counts(normalize=True)

In [ ]:
plot_var('term', 'Term', continuous=False)

About 76% of loans are 36 months, and the rest are 60 months. Loans with 60-month terms are less likely to be fully paid.

### int_rate

Data Dictionary: "Interest Rate on the loan."

In [ ]:
loans['int_rate'].describe()

Interest rates range from 5.3% to 30.9% (!), with a median of 13.1%.

In [ ]:
plot_var('int_rate', 'Interest Rate', continuous=True)

Charged-off loans tend to have higher interest rates.

### installment

Data Dictionary: "The monthly payment owed by the borrower if the loan originates."

In [ ]:
loans['installment'].describe()

Installments range from \$4.93 to \$1,714, with a median of \$377.

In [ ]:
plot_var('installment', 'Installment', continuous=True)

Charged-off loans tend to have slightly higher installments.

### grade, sub_grade

Data Dictionary for `grade`: "LendingClub assigned loan grade."

Data Dictionary for `sub_grade`: "LendingClub assigned loan subgrade."

What are the possible values of `grade` and `sub_grade`?

In [ ]:
print(sorted(loans['grade'].unique()))

In [ ]:
print(sorted(loans['sub_grade'].unique()))

The grade is implied by the subgrade, so let's drop the grade column.

In [ ]:
loans.drop('grade', axis=1, inplace=True)

In [ ]:
plot_var('sub_grade', 'Subgrade', continuous=False)

There's a clear trend of higher probability of charge-off as the subgrade worsens.

### emp_title

Data Dictionary: "The job title supplied by the Borrower when applying for the loan."

In [ ]:
loans['emp_title'].describe()

In [ ]:
loans.shape[0]

There are too many different job titles for this feature to be useful, so we drop it.

In [ ]:
loans.drop(labels='emp_title', axis=1, inplace=True)

### emp_length

Data Dictionary: "Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years." The actual data does not match this description:

In [ ]:
loans['emp_length'].value_counts(dropna=False).sort_index()

Note there are 42,253 loans without data on the length of employment.

Convert `emp_length` to integers:

In [ ]:
loans['emp_length'].replace(to_replace='10+ years', value='10 years', inplace=True)

In [ ]:
loans['emp_length'].replace('< 1 year', '0 years', inplace=True)

In [ ]:
def emp_length_to_int(s):
    if pd.isnull(s):
        return s
    else:
        return np.int8(s.split()[0])

In [ ]:
loans['emp_length'] = loans['emp_length'].apply(emp_length_to_int)

In [ ]:
loans['emp_length'].value_counts(dropna=False).sort_index()

In [ ]:
plot_var('emp_length', 'Employment length', continuous=False)

Loan status does not appear to depend much at all on employment length.

### home_ownership

Data Dictionary: "The home ownership status provided by the borrower during registration or obtained from the credit report. Our values are: RENT, OWN, MORTGAGE, OTHER."

In [ ]:
loans['home_ownership'].value_counts(dropna=False)

Replace ANY and NONE with OTHER, so that we have enough observations of each category.

In [ ]:
loans['home_ownership'].replace(['NONE', 'ANY'], 'OTHER', inplace=True)

In [ ]:
loans['home_ownership'].value_counts(dropna=False)

In [ ]:
plot_var('home_ownership', 'Home Ownership', continuous=False)

There appear to be small differences in charge-off rates by home ownership status. Renters have slightly less probability of fully paying off the loan. Because of the large numbers of observations in each category, these differences are statistically significant.

### annual_inc

Data Dictionary: "The self-reported annual income provided by the borrower during registration."

In [ ]:
loans['annual_inc'].describe()

Annual income ranges from \$0 to \$9,550,000, with a median of \$65,000. (Note this is more than double the national median individual income in the US.)

Because of the large range of incomes, we should take a log transform of the annual income.

In [ ]:
loans['annual_inc'] = loans['annual_inc'].apply(lambda x: np.log10(x+1))

In [ ]:
loans['annual_inc'].describe()

In [ ]:
plot_var('annual_inc', 'Log Annual Income', continuous=True)

It appears that individuals with somewhat higher income are more likely to pay off their loans. Let's break this down:

In [ ]:
loans.groupby('loan_status')['annual_inc'].describe()

There is a small, but likely statistically significant difference in the medians.

### verification_status

Data Dictionary: "Indicates if income was verified by [Lending Club], not verified, or if the income source was verified."

In [ ]:
loans['verification_status'].value_counts(dropna=False)

In [ ]:
plot_var('verification_status', 'Verification Status', continuous=False)

### issue_d

Data Dictionary: "The month which the loan was funded."

Because we're only using variables available before the loan was funded, `issue_d` will not be included in the final model. We're keeping it for now just to perform the train/test split later, then we'll drop it.

### purpose

Data Dictionary: "A category provided by the borrower for the loan request."

In [ ]:
loans['purpose'].value_counts()

Calculate the fully-paid rates by purpose:

In [ ]:
loans.groupby('purpose')['loan_status'].value_counts(normalize=True).loc[:,'Charged Off'].sort_values(ascending=False)

We see that, for example, 87.8% of loans for weddings are fully paid, but only 69.7% of small business loans are fully paid.

### title

Data Dictionary: "The loan title provided by the borrower."

In [ ]:
loans['title'].sample(5)

In [ ]:
loans['title'].describe()

View the top 10 loan titles, and their frequencies:

In [ ]:
loans['title'].value_counts().head(10)

There are 60,298 different purposes in the dataset, and based on the top 10 titles, the `purpose` variable appears to already contain this information. So we drop the `title` variable.

In [ ]:
loans.drop(labels='title', axis=1, inplace=True)

### zip_code, addr_state

Data Dictionary for `zip_code`: "The first 3 numbers of the zip code provided by the borrower in the loan application."

Data Dictionary for `addr_state`: "The state provided by the borrower in the loan application."

In [ ]:
loans['zip_code'].sample(5)

In [ ]:
loans['zip_code'].nunique()

In [ ]:
loans['addr_state'].sample(5)

In [ ]:
loans['addr_state'].nunique()

There are a lot of different zip codes, so let's just retain the state column.

In [ ]:
loans.drop(labels='zip_code', axis=1, inplace=True)

Calculate the fully-paid rates by address state:

In [ ]:
loans.groupby('addr_state')['loan_status'].value_counts(normalize=True).loc[:,'Charged Off'].sort_values(ascending=False)

The percentages of loans fully paid ranges from 72.4% in Mississippi to 86.9% in Washington, DC.

### dti

Data Dictionary: "A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income."

In [ ]:
loans['dti'].describe()

Note sure if the values of -1 and 999 make sense...

There are several outliers that mess up our default plots. Plot a histogram for `dti` less than 60:

In [ ]:
plt.figure(figsize=(8,3), dpi=90)
sns.distplot(loans.loc[loans['dti'].notnull() & (loans['dti']<60), 'dti'], kde=False)
plt.xlabel('Debt-to-income')
plt.ylabel('Count')
plt.title('Debt-to-income')

How many of the `dti` values are "outliers" (above 60)?

In [ ]:
(loans['dti']>=60).sum()

Very few.

Compare the `dti` medians for fully paid and charged-off loans:

In [ ]:
loans.groupby('loan_status')['dti'].median()

Borrowers who charge off tend to have higher debt-to-income ratios.

### earliest_cr_line

Data Dictionary: "The month the borrower's earliest reported credit line was opened."

In [ ]:
loans['earliest_cr_line'].sample(5)

In [ ]:
loans['earliest_cr_line'].isnull().any()

Let's just retain the year for simplicity:

In [ ]:
loans['earliest_cr_line'] = loans['earliest_cr_line'].apply(lambda s: int(s[-4:]))

In [ ]:
loans['earliest_cr_line'].describe()

In [ ]:
plot_var('earliest_cr_line', 'Year of Earliest Credit Line', continuous=True)

Borrowers who charge off tend to have opened their first credit line more recently.

### fico_range_low, fico_range_high

Data Dictionary for `fico_range_low`: "The lower boundary range the borrower’s FICO at loan origination belongs to."

Data Dictionary for `fico_range_high`: "The upper boundary range the borrower’s FICO at loan origination belongs to."

In [ ]:
loans[['fico_range_low', 'fico_range_high']].describe()

Check the correlation between these values:

In [ ]:
loans[['fico_range_low','fico_range_high']].corr()

We only need to keep one of the FICO scores.

In [ ]:
loans.drop(labels='fico_range_low', axis=1, inplace=True)

In [ ]:
plot_var('fico_range_high', 'FICO Range High', continuous=True)

There is a noticeable difference in FICO scores between fully paid and charged-off loans.

### pub_rec

Data Dictionary: "Number of derogatory public records."

In [ ]:
loans['pub_rec'].value_counts()

Is there a difference in average public records between fully paid loans and charged-off loans?

In [ ]:
loans.groupby('loan_status')['pub_rec'].mean()

### initial_list_status

Data Dictionary: "The initial listing status of the loan. Possible values are – W, F." I'm not sure what this means.

In [ ]:
loans['initial_list_status'].value_counts()

In [ ]:
plot_var('initial_list_status', 'Initial List Status', continuous=False)

### policy_code

Data Dictionary: "publicly available policy_code=1; new products not publicly available policy_code=2"

In [ ]:
loans['policy_code'].value_counts()

All values are the same; drop this variable.

In [ ]:
loans.drop(labels='policy_code', axis=1, inplace=True)

### application_type

Data Dictionary: "Indicates whether the loan is an individual application or a joint application with two co-borrowers."

In [ ]:
loans['application_type'].value_counts()

In [ ]:
plot_var('application_type', 'Application Type', continuous=False)

Joint loans are slightly more likely to be fully paid, but because there are few joint apps, it's unclear at the moment whether the difference is statistically significant.

### mort_acc

Data Dictionary: "Number of mortgage accounts."

In [ ]:
loans['mort_acc'].describe()

Not sure how someone can have 51 mortgage accounts...but apparently they do.

Top 10 values:

In [ ]:
loans['mort_acc'].value_counts().head(10)

In [ ]:
plot_var('mort_acc', 'Number of Mortgage Accounts', continuous=True)

Compare the averages by loan status:

In [ ]:
loans.groupby('loan_status')['mort_acc'].mean()

Individuals who pay off their loans tend to have more mortgage accounts on average.

### pub_rec_bankruptcies

Data Dictionary: "Number of public record bankruptcies."

In [ ]:
loans['pub_rec_bankruptcies'].value_counts().sort_index()

In [ ]:
plot_var('pub_rec_bankruptcies', 'Public Record Bankruptcies', continuous=False)

# More Pre-processing

## Convert Loan Status to Binary

Change the response variable `loan_status` to a 0/1 variable, where 0 indicates fully paid and 1 indicates charge-off:

In [ ]:
loans['charged_off'] = (loans['loan_status'] == 'Charged Off').apply(np.uint8)
loans.drop('loan_status', axis=1, inplace=True)

## Create Dummy Variables

How many variables do we currently have?

In [ ]:
loans.shape[1]

Which variables have missing values?

In [ ]:
missing_fractions = loans.isnull().mean().sort_values(ascending=False)

In [ ]:
missing_fractions[missing_fractions > 0]

There are no categorical variables with missing values, and therefore we don't need any `NaN` dummy variables.

Create dummy variables for the categorical variables:

In [ ]:
loans = pd.get_dummies(loans, columns=['sub_grade', 'home_ownership', 'verification_status', 'purpose', 'addr_state', 'initial_list_status', 'application_type'], drop_first=True)

In [ ]:
loans.shape

Check our data with the new dummy variables:

In [ ]:
loans.sample(3)

Looks good!

## Train/test split

We'll make our predictions more realistic by performing the train/test split based on the month that the loan was funded. That is, we'll use loans funded on earlier dates to predict the outcome of loans funded on later dates.

The variable `issue_d` includes the month which the loan was funded. Let's peek at this variable:

In [ ]:
loans['issue_d'].sample(5)

So `issue_d` only includes month and year data. Are there any missing values?

In [ ]:
loans['issue_d'].isnull().any()

No. Let's convert the issue dates to datetime objects:

In [ ]:
loans['issue_d'] = pd.to_datetime(loans['issue_d'])

In [ ]:
loans['issue_d'].sample(5)

The `issue_d` values got converted to dates, all on the first day of the month. Check summary statistics of the issue dates:

In [ ]:
loans['issue_d'].describe()

Note there are only 124 unique issue dates over the 10-year period because we only have month/year information. In this particular dataset, the first loans were issued in June 2007, and the most recent loans were issued in September 2017. The busiest month was October 2014 with 33,699 loans funded in that month. What is the distribution of loans funded per year?

In [ ]:
plt.figure(figsize=(6,3), dpi=90)
loans['issue_d'].dt.year.value_counts().sort_index().plot.bar(color='darkblue')
plt.xlabel('Year')
plt.ylabel('Number of loans funded')
plt.title('Loans Funded per Year')

We'll form the test set from the most recent 10% of the loans.

In [ ]:
loans_train = loans.loc[loans['issue_d'] <  loans['issue_d'].quantile(0.9)]
loans_test =  loans.loc[loans['issue_d'] >= loans['issue_d'].quantile(0.9)]

Check that we properly partitioned the loans:

In [ ]:
print('Number of loans in the partition:   ', loans_train.shape[0] + loans_test.shape[0])
print('Number of loans in the full dataset:', loans.shape[0])

The partition looks good, so we can delete the original `loans` dataframe.

In [ ]:
del loans

Let's look at the summary statistics of the issue dates in the train/validation/test sets:

In [ ]:
loans_train['issue_d'].describe()

In [ ]:
loans_test['issue_d'].describe()

The training set includes loans from June 2007 to June 2016. The test set includes loans from July 2016 to September 2017.

Now we can delete the `issue_d` variable, because it was not available before the loan was funded.

In [ ]:
loans_train.drop('issue_d', axis=1, inplace=True)
loans_test.drop('issue_d', axis=1, inplace=True)

Now separate the predictor variables from the response variable:

In [ ]:
X_train = loans_train.drop('charged_off', axis=1)
y_train = loans_train['charged_off']
X_test = loans_test.drop('charged_off', axis=1)
y_test = loans_test['charged_off']

In [ ]:
del loans_train, loans_test

## Check missing data

How complete is our training data?

In [ ]:
missing_fractions = X_train.isnull().mean().sort_values(ascending=False)

In [ ]:
missing_fractions[missing_fractions > 0]

As part of a machine learning pipeline, we'll perform mean imputation of the missing data.

# Linear Correlations with Loan Status

In [ ]:
linear_dep = pd.DataFrame()

In [ ]:
for col in X_train.columns:
    linear_dep.loc[col, 'corr'] = X_train[col].corr(y_train)
linear_dep['abs_corr'] = abs(linear_dep['corr'])

In [ ]:
from sklearn.feature_selection import f_classif

In [ ]:
for col in X_train.columns:
    mask = X_train[col].notnull()
    linear_dep.loc[col, 'F'], linear_dep.loc[col, 'p'] = f_classif(pd.DataFrame(X_train.loc[mask, col]), y_train.loc[mask])

In [ ]:
linear_dep.sort_values('abs_corr', ascending=True, inplace=True)

In [ ]:
linear_dep.iloc[-20:, [0,2,3]]

# Model Training and Testing

## Logistic regression with stochastic gradient descent

The SGDClassifier implements linear classifiers (SVM, logistic regression, a.o.) with SGD training. The linear classifier is chosen by the `loss` hyperparameter.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.linear_model import SGDClassifier

In [ ]:
pipeline_sgdlogreg = Pipeline([
    ('imputation', Imputer(copy=False)),
    ('standardization', StandardScaler(copy=False)),
    ('classifier', SGDClassifier(loss='log', penalty='l2', max_iter=1000, tol=1e-3, random_state=1, warm_start=True))
])

In [ ]:
from sklearn.model_selection import cross_val_score
cvscores_sgdlogreg = cross_val_score(estimator=pipeline_sgdlogreg, X=X_train, y=y_train, scoring='roc_auc', cv=5, n_jobs=-1, pre_dispatch='2*n_jobs')

In [ ]:
print(cvscores_sgdlogreg)
print(cvscores_sgdlogreg.mean())

## Random forest classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
pipeline_rfc = Pipeline([
    ('imputation', Imputer(copy=False)),
    ('standardization', StandardScaler(copy=False)),
    ('classifier', RandomForestClassifier(n_estimators=20, n_jobs=-1, random_state=1))
])

In [ ]:
cvscores_rfc = cross_val_score(estimator=pipeline_rfc, X=X_train, y=y_train, scoring='roc_auc', cv=5, n_jobs=-1, pre_dispatch='2*n_jobs')

In [ ]:
print(cvscores_rfc)
print(cvscores_rfc.mean())

## Tune hyperparameters of the chosen model

It looks like logistic regression performs best, so we'll tune the hyperparameters of logistic regression using 5-fold cross validation.

In [ ]:
param_grid = {
    'classifier__alpha': [10**p for p in np.arange(-4, 3, 1)],
    'classifier__penalty': ['l1', 'l2']
}

print(param_grid)

Instantiate the grid estimator. We'll use the Matthews correlation coefficient as our scoring metric.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
grid_sgdlogreg = GridSearchCV(estimator=pipeline_sgdlogreg, param_grid=param_grid, scoring='roc_auc', n_jobs=-1, pre_dispatch='2*n_jobs', cv=5, verbose=1, return_train_score=False)

Run the grid search (this could take some time).

In [ ]:
grid_sgdlogreg.fit(X_train, y_train)

View full results of the grid search:

In [ ]:
pd.DataFrame(grid_sgdlogreg.cv_results_)

Hyperparameters that gave the best results on the hold out data:

In [ ]:
grid_sgdlogreg.best_params_

Mean cross-validated AUCROC score of the best estimator:

In [ ]:
grid_sgdlogreg.best_score_

## Test set evaluation
<a id="8.2"></a>

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
y_score = grid_sgdlogreg.predict_proba(X_test)[:,1]
roc_auc_score(y_test, y_score)

# Conclusion